# Optimal Power Flow: IEEE 33 Bus Case with Pandapower
- Pyomo로 구성한 OPF 결과와 Pandapower OPF 결과 비교
- Pandapower 참고: https://dwightreid.com/site/power-system-contingency-analysis-with-python-pandapower/
- IEEE 33 bus 데이터 구조
  - https://matpower.org/docs/ref/matpower6.0/case33bw.html

1. 계통 불러오기

In [1]:
import pandapower as pp
import pandapower.networks as pn
import pandas as pd
import numpy as np

net = pn.case33bw()
#net.shunt['in_service'] = False
pp.runpp(net,numba=False)
base_MVA = net._ppc['baseMVA']


In [2]:
net.gen

,name,bus,p_mw,vm_pu,sn_mva,min_q_mvar,max_q_mvar,scaling,slack,in_service,slack_weight,type


In [3]:
net.sgen

,name,bus,p_mw,q_mvar,sn_mva,scaling,in_service,type,current_source


In [4]:
net.ext_grid

,name,bus,vm_pu,va_degree,slack_weight,in_service,max_p_mw,min_p_mw,max_q_mvar,min_q_mvar
0,None,0,1.0,0.0,1.0,True,10.0,0.0,10.0,-10.0


In [5]:
net.poly_cost

,element,et,cp0_eur,cp1_eur_per_mw,cp2_eur_per_mw2,cq0_eur,cq1_eur_per_mvar,cq2_eur_per_mvar2
0,0,ext_grid,0.0,20.0,0.0,0.0,0.0,0.0


2. 조류계산 후 Y 행렬 계산 

In [6]:
ymat = net._ppc['internal']['Ybus'].todense()
Y_mat_panda = pd.DataFrame(ymat)
if 0 in net.bus['name'].values:
    bus_index = net.bus['name'].values + 1
else:
    bus_index = net.bus['name'].values 
    
Y_mat_panda.index = bus_index
Y_mat_panda.columns = bus_index
Y_mat_panda.to_csv('./Pre_cal_data/Ymat_panda.csv')
Y_mat_panda.head(5)

,1,2,3,4,5,6,7,8,9,10,...,24,25,26,27,28,29,30,31,32,33
1,137.979749-70.336748j,-137.979749+70.336748j,0.0000000+0.0000000j,0.0000000+0.0000000j,0.0000000+0.0000000j,0.000000+0.000000j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,...,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j
2,-137.979749+70.336748j,214.943686-132.295494j,-25.813726+13.147722j,0.0000000+0.0000000j,0.0000000+0.0000000j,0.000000+0.000000j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,...,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j
3,0.0000000+0.0000000j,-25.8137260+13.1477220j,84.801838-47.403325j,-34.772102+17.709070j,0.0000000+0.0000000j,0.000000+0.000000j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,...,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j
4,0.0000000+0.0000000j,0.00000000+0.00000000j,-34.772102+17.709070j,68.165769-34.716971j,-33.393667+17.007900j,0.000000+0.000000j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,...,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j
5,0.0000000+0.0000000j,0.00000000+0.00000000j,0.0000000+0.0000000j,-33.393667+17.007900j,44.607113-26.687883j,-11.213446+9.679983j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,...,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j,0.0+0.0j


3. 복소수 성분을 제거하고 Pyomo에 맞는 Y 행렬 생성 

In [7]:
bus_multi_index = pd.MultiIndex.from_product(
    [bus_index, bus_index],
    names=["Bus_i", "Bus_j"]
)

Y_mat_pyomo = pd.DataFrame(index=bus_multi_index,columns=['Bus_G','Bus_B'])

for i in bus_index:
    for j in bus_index:
        Y_mat_pyomo.loc[(i,j),'Bus_G'] = np.real(Y_mat_panda.loc[i,j])
        Y_mat_pyomo.loc[(i,j),'Bus_B'] = np.imag(Y_mat_panda.loc[i,j])

Y_mat_pyomo.to_csv('./Pre_cal_data/Ymat_pyomo.csv')        
Y_mat_pyomo.head(5)  


Bus_G      Bus_B
Bus_i Bus_j                       
1     1      137.979749 -70.336748
      2     -137.979749  70.336748
      3             0.0        0.0
      4             0.0        0.0
      5             0.0        0.0

4. Pandapower에서 모선 데이터 불러오기 및 Pyomo 형태로 저장 

In [8]:
Bus_info = pd.DataFrame(net.bus[['name','vn_kv','in_service','max_vm_pu','min_vm_pu']])
if 0 in Bus_info['name'].values:
    Bus_info['name'] = bus_index

Bus_info.set_index('name',inplace=True)
Bus_info.index.name = 'Bus_i'
Bus_info.to_csv('./Pre_cal_data/Bus_info_pyomo.csv')
Bus_info.head(5)

,vn_kv,in_service,max_vm_pu,min_vm_pu
Bus_i,,,,
1,12.66,True,1.0,1.0
2,12.66,True,1.1,0.9
3,12.66,True,1.1,0.9
4,12.66,True,1.1,0.9
5,12.66,True,1.1,0.9


5. Pandapower에서 발전 데이터 불러오기 및 Pyomo 형태로 저장
   1. net.gen에서의 bus는 0부터 시작이므로 +1을 더해줌 (계통마다 확인 필요) 

In [9]:
net._ppc['gen'][0]

array([ 0.00000000e+00,  3.91767704e+00,  2.43514091e+00,  0.00000000e+00,
        0.00000000e+00,  1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
        1.00000000e+09, -1.00000000e+09,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  1.00000000e+00])

In [12]:
try:
    gen_info = pd.DataFrame(net.gen[['bus','vm_pu','min_q_mvar','max_q_mvar','in_service','max_p_mw','min_p_mw']])
    gen_info
except:
    print("Check genator info")


Check genator info


In [13]:
slack_info = pd.DataFrame(net.ext_grid[['bus','vm_pu','min_q_mvar','max_q_mvar','in_service','max_p_mw','min_p_mw']])
slack_info

,bus,vm_pu,min_q_mvar,max_q_mvar,in_service,max_p_mw,min_p_mw
0,0,1.0,-10.0,10.0,True,10.0,0.0


In [14]:
try:
    # 기본 발전 데이터
    gen_info = pd.DataFrame(net.gen[['bus','p_mw','vm_pu','min_q_mvar','max_q_mvar','in_service','max_p_mw','min_p_mw']])

    # 발전 비용함수 추가
    gen_info['cp0_eur']=net.poly_cost[net.poly_cost['et'] == 'gen'].reset_index(drop=True)['cp0_eur']
    gen_info['cp1_eur_per_mw']=net.poly_cost[net.poly_cost['et'] == 'gen'].reset_index(drop=True)['cp1_eur_per_mw']
    gen_info['cp2_eur_per_mw2']=net.poly_cost[net.poly_cost['et'] == 'gen'].reset_index(drop=True)['cp2_eur_per_mw2']

    gen_info['cq0_eur']=net.poly_cost[net.poly_cost['et'] == 'gen'].reset_index(drop=True)['cq0_eur']
    gen_info['cq1_eur_per_mvar']=net.poly_cost[net.poly_cost['et'] == 'gen'].reset_index(drop=True)['cq1_eur_per_mvar']
    gen_info['cq2_eur_per_mvar2']=net.poly_cost[net.poly_cost['et'] == 'gen'].reset_index(drop=True)['cq2_eur_per_mvar2']

    tmp = gen_info['bus'].values + 1
    gen_info['bus'] = tmp
    
except:
    print("Check genator info")

# Slack 모선 데이터 - Slack 모선이 발전기인 경우
slack_info = pd.DataFrame(net.ext_grid[['bus','vm_pu','min_q_mvar','max_q_mvar','in_service','max_p_mw','min_p_mw']])
slack_info['p_mw'] = 0
slack_info = slack_info[['bus','p_mw','vm_pu','min_q_mvar','max_q_mvar','in_service','max_p_mw','min_p_mw']]

# 발전 비용함수 추가
slack_info['cp0_eur']=net.poly_cost[net.poly_cost['et'] == 'ext_grid'].reset_index(drop=True)['cp0_eur']
slack_info['cp1_eur_per_mw']=net.poly_cost[net.poly_cost['et'] == 'ext_grid'].reset_index(drop=True)['cp1_eur_per_mw']
slack_info['cp2_eur_per_mw2']=net.poly_cost[net.poly_cost['et'] == 'ext_grid'].reset_index(drop=True)['cp2_eur_per_mw2']

slack_info['cq0_eur']=net.poly_cost[net.poly_cost['et'] == 'ext_grid'].reset_index(drop=True)['cq0_eur']
slack_info['cq1_eur_per_mvar']=net.poly_cost[net.poly_cost['et'] == 'ext_grid'].reset_index(drop=True)['cq1_eur_per_mvar']
slack_info['cq2_eur_per_mvar2']=net.poly_cost[net.poly_cost['et'] == 'ext_grid'].reset_index(drop=True)['cq2_eur_per_mvar2']

tmp = slack_info['bus'].values + 1
slack_info['bus']=tmp

try:
    gen_info = pd.concat([gen_info,slack_info])
    gen_info.sort_values(by=['bus'],axis=0,inplace=True)
    gen_info.reset_index(inplace=True,drop=True)
    gen_info.set_index('bus',inplace=True)
    gen_info.index.name = 'Bus_i'
except:
    gen_info = slack_info.copy()
    gen_info.reset_index(inplace=True,drop=True)
    gen_info.set_index('bus',inplace=True)
    gen_info.index.name = 'Bus_i'

gen_info.to_csv('./Pre_cal_data/Gen_info_pyomo.csv')
gen_info.head(5)


Check genator info


,p_mw,vm_pu,min_q_mvar,max_q_mvar,in_service,max_p_mw,min_p_mw,cp0_eur,cp1_eur_per_mw,cp2_eur_per_mw2,cq0_eur,cq1_eur_per_mvar,cq2_eur_per_mvar2
Bus_i,,,,,,,,,,,,,
1,0,1.0,-10.0,10.0,True,10.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0


6. Pandapower 에서 부하 데이터 불러오기 및 Pyomo 형태로 저장
   1. net.load에서의 bus는 0부터 시작이므로 +1을 더해줌 (계통마다 확인 필요) 

In [15]:
load_info = pd.DataFrame(net.load[['bus','p_mw','q_mvar','in_service']])
tmp = load_info['bus'].values + 1
load_info['bus'] = tmp
load_info.set_index('bus',inplace=True)
load_info.index.name = 'Bus_i'

load_info.to_csv('./Pre_cal_data/Load_info_pyomo.csv')
load_info.head(5)

,p_mw,q_mvar,in_service
Bus_i,,,
2,0.10,0.06,True
3,0.09,0.04,True
4,0.12,0.08,True
5,0.06,0.03,True
6,0.06,0.02,True


Pandapower OPF 결과

In [16]:
pp.runopp(net, delta=1e-16,numba=False)

gen_mw_total = net.res_gen['p_mw'].sum() 
imports_mw_total = net.res_ext_grid['p_mw'].sum()

print('total gen MW:', gen_mw_total + imports_mw_total)
print('total imported gen MW:', imports_mw_total)
print('total local gen MW:', gen_mw_total)
print('total load MW:', net.res_load['p_mw'].sum())

total gen MW: 3.917677126455767
total imported gen MW: 3.917677126455767
total local gen MW: 0.0
total load MW: 3.715


In [17]:
net

This pandapower network includes the following parameter tables:
   - bus (33 elements)
   - load (32 elements)
   - ext_grid (1 element)
   - line (37 elements)
   - poly_cost (1 element)
 and the following results tables:
   - res_bus (33 elements)
   - res_line (37 elements)
   - res_ext_grid (1 element)
   - res_load (32 elements)
 and the following result values:
   - res_cost